<a href="https://colab.research.google.com/github/choi-yh/DataStructure/blob/master/8_3_Minimum_Spanning_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 간선에 가중치가 존재하는 가중치 그래프에서 가중치의 합이 최소화되도록 구성한 트리를 말한다.  
* 최소 비용 신장 트리는 두 개의 정점 사이에 한 개 경로만 존재한다. 그러므로 싸이클도 존재할 수 없다.  
* 복잡한 그래프에서 최소 비용 신장 트리를 구성하는 것은 네트워크가 작동할 수 있는 최소 조건을 구하는 의미가 있다.  
* 최소 비용 신장 트리를 구하는 방법으로는 Kruskal 알고리즘, Prim 알고리즘, Sollin 알고리즘 등이 있다.

* Krusakl 알고리즘
    * 간선들의 가중값을 정렬한 후, 가장 작은 가중값을 가지는 간선부터 추가해 간다.
    * 추가 중에 간선의 수가 n-1이 되면 알고리즘이 종료된다. n은 노드의 수다.  

    <center><img src="https://drive.google.com/uc?id=1udy_0nAYtLq8XrPQt1O1uexRGsMwyS1V" width="500" height="500" ></center>

    * 위의 그림에서 최소 간선 (4, 6)부터 차례로 빈 트리에 삽입한다.  
    * 삽입 과정에서 (1, 4), (2, 4), (3, 6)은 싸이클이 형성되므로 스킵한다.
    * 삽입하다가 삽입된 간선이 6개(노드 수 -1)가 되면 알고리즘이 종료된다.  
    * 싸이클을 탐지하는 방법으로 **Union-Find** 알고리즘을 사용할 수 있다.  

* **Union Find Algorithm**
* Union Find 알고리즘은 노드 연결을 나타내는 리스트를 정의하고, 추가되는 엣지 (n1, n2)에 대해 Union 해간다.
* 예를 들어, 현재 [[0,1,2], [3,4]] 이 존재할 때, 아래와 같이 처리한다.  
    * [[0,1,2], [3,4]] + [5,6] = [[0,1,2], [3,4], [5,6]] # *연결할 수 없으므로 따로 생성*
    * [[0,1,2], [3,4]] + [1,5] = [[0,1,2,5], [3,4] # *존재하는 곳에 연결*
    * [[0,1,2], [3,4]] + [1,3] = [[0,1,2,3,4]] # *n1, n2 모두 존재하므로 두 개가 존재하는 서브 리스트를 합친다.*
    * [[0,1,2], [3,4]] + [0,2] = 싸이클 # *0과 2가 모두 0번째로 같은 위치에 존재하므로 추가할 필요가 없고 이때 싸이클이 존재한다.*

In [0]:
def isCycle(u, n):
    idx1 = idx2 = -1
    n1 = n[0]
    n2 = n[1]
    for i in range(len(u)):
        if n1 in u[i]:
            idx1 = i
        if n2 in u[i]:
            idx2 = i

    if idx1 == -1 and idx2 == -1: # [n1, n2]가 기존 리스트에 없다면
        u.append([n1, n2])
        print(u)
        return False
    elif idx1 == -1: # [n1, n2]가 기존 리스트 중 한 곳에만 있다면
        u[idx2].append(n1)
        print(u)
        return False
    elif idx2 == -1: # [n1, n2]가 기존 리스트 중 한 곳에만 있다면
        u[idx1].append(n2)
        print(u)
        return False
    elif idx1 != idx2: # [n1, n2]가 기존 리스트에 서로 다른 곳에 존재한다면
        d1 = u[idx1]
        d2 = u[idx2]
        union = d1 + d2
        u.remove(d1)
        u.remove(d2)
        u.append(union)
        print(u)
        return False
    elif idx1 == idx2 and len(u[idx1]) > 2: # [n1, n2]가 기존 리스트에 같은 곳에 존재한다면
        print(u)
        return True

In [0]:
u = [[0, 1, 2],[3, 4]]
print(isCycle(u, (2,3)))

u = [[0, 1, 2],[3, 4]]
print(isCycle(u, (0,2)))

u = [[0, 1, 2],[3, 4]]
print(isCycle(u, (1,5)))

u = [[0, 1, 2],[3, 4]]
print(isCycle(u, (4,6)))

[[0, 1, 2, 3, 4]]
False
[[0, 1, 2], [3, 4]]
True
[[0, 1, 2, 5], [3, 4]]
False
[[0, 1, 2], [3, 4, 6]]
False


In [0]:
class SpanningTree:
    def __init__(self, graph):
        self.graph = graph
        self.nodes = set()
        self.union = [] # union-find 알고리즘에서 기존 엣지 리스트

        # 그래프의 노드를 구함
        for edge in graph:
            self.nodes.add(edge[0])
            self.nodes.add(edge[1])
        self.nNode = len(self.nodes)

    def isCycle(self, n1, n2):
        idx1 = idx2 = -1
        for i in range(len(self.union)):
            if n1 in self.union[i]:
                idx1 = i
            if n2 in self.union[i]:
                idx2 = i

        if idx1 == -1 and idx2 == -1: # [n1, n2]가 기존 리스트에 없다면
            self.union.append([n1, n2])
            return False
        elif idx1 == -1: # [n1, n2]가 기존 리스트 중 한 곳에만 있다면
            self.union[idx2].append(n1)
            return False
        elif idx2 == -1: # [n1, n2]가 기존 리스트 중 한 곳에만 있다면
            self.union[idx1].append(n2)
            return False
        elif idx1 != idx2: # [n1, n2]가 기존 리스트에 서로 다른 곳에 존재한다면
            d1 = self.union[idx1]
            d2 = self.union[idx2]
            union = d1 + d2
            self.union.remove(d1)
            self.union.remove(d2)
            self.union.append(union)
            print(u)
            return False
        elif idx1 == idx2 and len(self.union[idx1]) > 2: # [n1, n2]가 기존 리스트에 같은 곳에 존재한다면
            print(self.union[idx1])
            return True

    def kruskal(self):
        self.graph.sort(key = lambda t: t[2]) # 그래프를 2번째 원소 기준으로 소트
        tree = [] # MST 담을 리스트
        nedges = 0 # nedges == self.nNode - 1이면 알고리즘이 끝난다.
        i = 0
        while nedges < self.nNode - 1:
            if self.isCycle(self.graph[i][0], self.graph[i][1]) == False:
                tree.append(self.graph[i])
                nedges += 1
            else:
                print("싸이클 발견", self.graph[i])
            i += 1
        return tree

In [0]:
g = [(0,1,9), (0,2,10), (1,3,10), (1,4,5), (1,6,3), (2,3,9), (2,4,7), (2,5,2), (3,5,4), (3,6,8), (4,6,1), (5,6,6)]
t = SpanningTree(g)
print(t.kruskal())

[4, 6, 1]
싸이클 발견 (1, 4, 5)
[[0, 1, 2], [3, 4, 6]]
[2, 5, 3, 4, 6, 1]
싸이클 발견 (2, 4, 7)
[2, 5, 3, 4, 6, 1]
싸이클 발견 (3, 6, 8)
[(4, 6, 1), (2, 5, 2), (1, 6, 3), (3, 5, 4), (5, 6, 6), (0, 1, 9)]
